<a href="https://colab.research.google.com/github/katya123123/ML-labs/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22ML_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.cluster import DBSCAN

In [2]:
df = pd.read_csv("/content/student-mat.csv")
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


Категориальная переменная - sex (Y)

Числовая переменная для анализа - absences (X)

In [3]:
describe = df['absences'].describe()
print(describe)

count    395.000000
mean       5.708861
std        8.003096
min        0.000000
25%        0.000000
50%        4.000000
75%        8.000000
max       75.000000
Name: absences, dtype: float64


In [4]:
agg = df.groupby(['sex']).agg({'absences': ['mean', 'median', 'min', 'max', np.std, pd.Series.mode]})
print(agg)

     absences                              
         mean median min max       std mode
sex                                        
F    6.216346    4.0   0  75  9.445819    0
M    5.144385    4.0   0  38  5.980749    0


In [5]:
q1 = df.groupby(['sex'])['absences'].quantile(0.25)
print(q1)

sex
F    0.0
M    0.0
Name: absences, dtype: float64


In [6]:
q2 = df.groupby(['sex'])['absences'].quantile(0.5)
print(q2)

sex
F    4.0
M    4.0
Name: absences, dtype: float64


In [7]:
q3 = df.groupby(['sex'])['absences'].quantile(0.75)
print(q3)

sex
F    8.0
M    8.0
Name: absences, dtype: float64


In [8]:
iqr = q3 - q1
print(iqr)


sex
F    8.0
M    8.0
Name: absences, dtype: float64


**Выводы:**

1. Среднее в обоих группах приерно равно, различие составляет < 1
2. СРеднее отклонение отличается сильнее, чем среднее. Отклонение отличается примерно на 4
3. Однако, несмотря на то что отклонение в группе F больше, но и максимальное значение больше. Между значениями 75 и 0 среднее отклонение 9.5 примерно соотносимо со средним отклонением 5.4 между 38 и 0.
4. Медианные значения полностью совпадают, учитывая, что это более объективная переменная для анализа, чем среднее, группы имеют примерно одинаковые разбросы данных.
5. Квантили во обоих группах абсолютно одинаковые, что подтверждает предыдущее предположение.
6. Учитывая, что медиана и квантили в обоих группах одинаковые, но максимальные значения очень сильно отличаются, можно сделать вывод, что в первой группе есть сильный (или сильные) выбросы.
7. Таким образом, среднее в группе F больше, поскольку на него оказывает влияние выброс (выбросы)


In [22]:
import plotly.express as plt
%matplotlib inline

In [23]:
fig = plt.histogram(x = df['absences'], y = df['sex'])
fig.show()

In [11]:
fig = plt.box(df, y='absences')
fig.show()

In [12]:
fig = plt.box(df, x = 'sex', y = 'absences')
fig.show()

In [13]:
fig = plt.scatter(df, x = 'G1', y = 'absences')
fig.show()

In [14]:
corr_matrix = df[['age', 'absences', 'G1']].corr()
fig = plt.imshow(corr_matrix)
fig.show()

#Примечание: мне показалось, что по двум числовым
#переменным - это совершенно не показательно и ни о чем не скажет, поэтому взяла хотя бы три

Судя по полученной матрице корреляции на количество пропусков большее влияние окащывает возраст, чем показатель G1. Помимо этого, по box графику видно, что выборка имеет сильные выбросы. Большинство выбросов приходится на категорию F, что подтверждает предположения, сделанные на основе описательной статистики.

**Задание для гениев**

In [15]:
#На основе среднего и отклонения
mean = df['absences'].mean()
median = df['absences'].median()
std = df['absences'].std()
N = df['absences'].count()
print("mean: ", mean, "median: ", median)
print("Выборка: ")
if median - mean <= 3 * math.sqrt(std**2/N):
  print("симметричная")
  cof = 3
else:
  print("несимметричная")
  cof = 5

mean:  5.708860759493671 median:  4.0
Выборка: 
симметричная


In [16]:
class Outline:
  def __init__(self, df):
    self.df = df

  def del_out_by_mean(self, col):
    arr =[]
    for i in self.df[col]:
      self.df[col].drop(index = i)
      mean = self.df[col].mean()
      std = self.df[col].std()
      min = mean - cof * std
      max = mean + cof * std
      if (i >= min) and (i <= max):
        arr.append(i)

    return arr

  def del_out_by_shovene(self, col):
    arrSh = []
    n = self.df[col].count()
    for i in self.df[col]:
      meanSh = self.df[col].mean()
      stdSh = self.df[col].std()
      if math.erfc((abs(i-meanSh))/stdSh) > 1/(2*n):
        arrSh.append(i)
      else:
        n -= 1
        self.df[col].drop(index = i)
    return arrSh

  def del_out_by_qu(self, col):
    arrQ = []
    Q1 = df['absences'].quantile(0.25)
    Q2 = df['absences'].quantile(0.5)
    Q3 = df['absences'].quantile(0.75)
    min = Q1-1.5*(Q3-Q1)
    max = Q3+1.5*(Q3-Q1)
    for i in df['absences']:
      if (i >= min) and (i <= max):
        arrQ.append(i)
    return arrQ

  def del_out_by_two(self, col):
    new_df = df[['absences', 'age']]
    cluster = DBSCAN(eps = 3, min_samples = 3).fit(new_df)
    new_df['labels'] = cluster.labels_
    idx = new_df[new_df['labels'] == -1].index
    new_df = new_df[new_df['labels'] != -1]
    fig = plt.scatter(new_df, x = 'absences', y = 'age', color = 'labels')
    fig.show()


In [17]:

meanOutDf = Outline(df)
meanOutDf.del_out_by_mean("absences")

[6,
 4,
 10,
 2,
 4,
 10,
 0,
 6,
 0,
 0,
 0,
 4,
 2,
 2,
 0,
 4,
 6,
 4,
 16,
 4,
 0,
 0,
 2,
 0,
 2,
 14,
 2,
 4,
 4,
 16,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 7,
 2,
 8,
 25,
 8,
 2,
 0,
 14,
 8,
 12,
 4,
 2,
 2,
 2,
 2,
 6,
 0,
 6,
 8,
 0,
 4,
 2,
 2,
 6,
 6,
 4,
 2,
 0,
 2,
 4,
 4,
 2,
 12,
 0,
 0,
 2,
 2,
 6,
 8,
 0,
 2,
 12,
 2,
 4,
 10,
 4,
 2,
 6,
 4,
 4,
 12,
 18,
 0,
 4,
 4,
 0,
 6,
 2,
 2,
 2,
 6,
 0,
 14,
 0,
 4,
 26,
 0,
 10,
 8,
 2,
 6,
 4,
 6,
 0,
 6,
 10,
 8,
 2,
 2,
 0,
 20,
 6,
 2,
 6,
 2,
 18,
 0,
 0,
 0,
 2,
 0,
 8,
 0,
 0,
 12,
 16,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 6,
 8,
 0,
 0,
 2,
 8,
 6,
 2,
 4,
 0,
 6,
 0,
 2,
 0,
 16,
 4,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 4,
 4,
 2,
 4,
 10,
 4,
 10,
 2,
 0,
 14,
 12,
 2,
 0,
 6,
 4,
 10,
 0,
 12,
 8,
 0,
 0,
 4,
 8,
 24,
 0,
 2,
 6,
 4,
 18,
 6,
 28,
 5,
 10,
 6,
 6,
 10,
 13,
 0,
 15,
 12,
 2,
 22,
 13,
 3,
 4,
 2,
 0,
 2,
 0,
 0,
 16,
 10,
 2,
 14,
 10,
 14,
 4,
 14,
 2,
 18,
 10,
 4,
 20,
 2,
 0,
 

In [18]:
arr =[]
for i in df['absences']:
  df['absences'].drop(index = i)
  mean = df['absences'].mean()
  std = df['absences'].std()
  min = mean - cof * std
  max = mean + cof * std
  if (i >= min) and (i <= max):
    arr.append(i)
print("Очищенный датафрейм: ", arr)
fig = plt.box(y = arr)
fig.show()

Очищенный датафрейм:  [6, 4, 10, 2, 4, 10, 0, 6, 0, 0, 0, 4, 2, 2, 0, 4, 6, 4, 16, 4, 0, 0, 2, 0, 2, 14, 2, 4, 4, 16, 0, 0, 0, 0, 0, 0, 2, 7, 2, 8, 25, 8, 2, 0, 14, 8, 12, 4, 2, 2, 2, 2, 6, 0, 6, 8, 0, 4, 2, 2, 6, 6, 4, 2, 0, 2, 4, 4, 2, 12, 0, 0, 2, 2, 6, 8, 0, 2, 12, 2, 4, 10, 4, 2, 6, 4, 4, 12, 18, 0, 4, 4, 0, 6, 2, 2, 2, 6, 0, 14, 0, 4, 26, 0, 10, 8, 2, 6, 4, 6, 0, 6, 10, 8, 2, 2, 0, 20, 6, 2, 6, 2, 18, 0, 0, 0, 2, 0, 8, 0, 0, 12, 16, 0, 0, 0, 0, 0, 0, 0, 8, 2, 2, 0, 0, 0, 2, 0, 0, 0, 6, 8, 0, 0, 2, 8, 6, 2, 4, 0, 6, 0, 2, 0, 16, 4, 0, 0, 0, 0, 2, 0, 0, 4, 4, 2, 4, 10, 4, 10, 2, 0, 14, 12, 2, 0, 6, 4, 10, 0, 12, 8, 0, 0, 4, 8, 24, 0, 2, 6, 4, 18, 6, 28, 5, 10, 6, 6, 10, 13, 0, 15, 12, 2, 22, 13, 3, 4, 2, 0, 2, 0, 0, 16, 10, 2, 14, 10, 14, 4, 14, 2, 18, 10, 4, 20, 2, 0, 14, 2, 0, 0, 0, 6, 4, 16, 8, 0, 0, 6, 4, 0, 0, 2, 6, 12, 8, 0, 21, 2, 1, 4, 0, 13, 2, 8, 10, 0, 15, 4, 2, 2, 2, 6, 22, 15, 8, 19, 1, 4, 4, 2, 5, 6, 6, 9, 11, 0, 12, 6, 8, 4, 0, 10, 0, 5, 14, 0, 0, 0, 20, 8, 0, 0, 18,

In [19]:
#Критерий Шовене
arrSh = []
n = df['absences'].count()
for i in df['absences']:
  meanSh = df['absences'].mean()
  stdSh = df['absences'].std()
  if math.erfc((abs(i-meanSh))/stdSh) > 1/(2*n):
    arrSh.append(i)
  else:
    n -= 1
    df['absences'].drop(index = i)
print("Очищенный датафрейм: ", arrSh)
fig = plt.box(y = arrSh)
fig.show()

Очищенный датафрейм:  [6, 4, 10, 2, 4, 10, 0, 6, 0, 0, 0, 4, 2, 2, 0, 4, 6, 4, 16, 4, 0, 0, 2, 0, 2, 14, 2, 4, 4, 16, 0, 0, 0, 0, 0, 0, 2, 7, 2, 8, 8, 2, 0, 14, 8, 12, 4, 2, 2, 2, 2, 6, 0, 6, 8, 0, 4, 2, 2, 6, 6, 4, 2, 0, 2, 4, 4, 2, 12, 0, 0, 2, 2, 6, 8, 0, 2, 12, 2, 4, 10, 4, 2, 6, 4, 4, 12, 18, 0, 4, 4, 0, 6, 2, 2, 2, 6, 0, 14, 0, 4, 0, 10, 8, 2, 6, 4, 6, 0, 6, 10, 8, 2, 2, 0, 20, 6, 2, 6, 2, 18, 0, 0, 0, 2, 0, 8, 0, 0, 12, 16, 0, 0, 0, 0, 0, 0, 0, 8, 2, 2, 0, 0, 0, 2, 0, 0, 0, 6, 8, 0, 0, 2, 8, 6, 2, 4, 0, 6, 0, 2, 0, 16, 4, 0, 0, 0, 0, 2, 0, 0, 4, 4, 2, 4, 10, 4, 10, 2, 0, 14, 12, 2, 0, 6, 4, 10, 0, 12, 8, 0, 0, 4, 8, 0, 2, 6, 4, 18, 6, 5, 10, 6, 6, 10, 13, 0, 15, 12, 2, 22, 13, 3, 4, 2, 0, 2, 0, 0, 16, 10, 2, 14, 10, 14, 4, 14, 2, 18, 10, 4, 20, 2, 0, 14, 2, 0, 0, 0, 6, 4, 16, 8, 0, 0, 6, 4, 0, 0, 2, 6, 12, 8, 0, 21, 2, 1, 4, 0, 13, 2, 8, 10, 0, 15, 4, 2, 2, 2, 6, 22, 15, 8, 19, 1, 4, 4, 2, 5, 6, 6, 9, 11, 0, 12, 6, 8, 4, 0, 10, 0, 5, 14, 0, 0, 0, 20, 8, 0, 0, 18, 0, 20, 3, 22, 1

In [20]:
#С помощью квартилей
arrQ = []
Q1 = df['absences'].quantile(0.25)
Q2 = df['absences'].quantile(0.5)
Q3 = df['absences'].quantile(0.75)
min = Q1-1.5*(Q3-Q1)
max = Q3+1.5*(Q3-Q1)
for i in df['absences']:
  if (i >= min) and (i <= max):
    arrQ.append(i)
print("Очищенный датафрейм: ", arrQ)
fig = plt.box(y = arrQ)
fig.show()

Очищенный датафрейм:  [6, 4, 10, 2, 4, 10, 0, 6, 0, 0, 0, 4, 2, 2, 0, 4, 6, 4, 16, 4, 0, 0, 2, 0, 2, 14, 2, 4, 4, 16, 0, 0, 0, 0, 0, 0, 2, 7, 2, 8, 8, 2, 0, 14, 8, 12, 4, 2, 2, 2, 2, 6, 0, 6, 8, 0, 4, 2, 2, 6, 6, 4, 2, 0, 2, 4, 4, 2, 12, 0, 0, 2, 2, 6, 8, 0, 2, 12, 2, 4, 10, 4, 2, 6, 4, 4, 12, 18, 0, 4, 4, 0, 6, 2, 2, 2, 6, 0, 14, 0, 4, 0, 10, 8, 2, 6, 4, 6, 0, 6, 10, 8, 2, 2, 0, 20, 6, 2, 6, 2, 18, 0, 0, 0, 2, 0, 8, 0, 0, 12, 16, 0, 0, 0, 0, 0, 0, 0, 8, 2, 2, 0, 0, 0, 2, 0, 0, 0, 6, 8, 0, 0, 2, 8, 6, 2, 4, 0, 6, 0, 2, 0, 16, 4, 0, 0, 0, 0, 2, 0, 0, 4, 4, 2, 4, 10, 4, 10, 2, 0, 14, 12, 2, 0, 6, 4, 10, 0, 12, 8, 0, 0, 4, 8, 0, 2, 6, 4, 18, 6, 5, 10, 6, 6, 10, 13, 0, 15, 12, 2, 13, 3, 4, 2, 0, 2, 0, 0, 16, 10, 2, 14, 10, 14, 4, 14, 2, 18, 10, 4, 20, 2, 0, 14, 2, 0, 0, 0, 6, 4, 16, 8, 0, 0, 6, 4, 0, 0, 2, 6, 12, 8, 0, 2, 1, 4, 0, 13, 2, 8, 10, 0, 15, 4, 2, 2, 2, 6, 15, 8, 19, 1, 4, 4, 2, 5, 6, 6, 9, 11, 0, 12, 6, 8, 4, 0, 10, 0, 5, 14, 0, 0, 0, 20, 8, 0, 0, 18, 0, 20, 3, 14, 0, 9, 0, 2, 1

In [21]:
#По двум переменным

new_df = df[['absences', 'age']]
cluster = DBSCAN(eps = 3, min_samples = 3).fit(new_df)
new_df['labels'] = cluster.labels_
idx = new_df[new_df['labels'] == -1].index
new_df = new_df[new_df['labels'] != -1]
fig = plt.scatter(new_df, x = 'absences', y = 'age', color = 'labels')
fig.show()
print("Индексы выбросов: ", idx)

<ipython-input-21-421c5523b550>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Индексы выбросов:  Int64Index([74, 183, 247, 276, 307, 315], dtype='int64')
